## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI, AzureOpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)

import os
endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
azure_key = os.getenv('AZURE_OPENAI_API_KEY')
deployment_chat = os.getenv('AZURE_OPENAI_DEPLOYMENT_CHAT')
api_version_chat = os.getenv('AZURE_OPENAI_API_VERSION_CHAT')
deployment_reasoning = os.getenv('AZURE_OPENAI_DEPLOYMENT_REASONING')
api_version_reasoning = os.getenv('AZURE_OPENAI_API_VERSION_REASONING')

openai_chat = AzureOpenAI(
    api_version=api_version_chat,
    azure_endpoint=endpoint,
    api_key=azure_key,
)

openai_reasoning = AzureOpenAI(
    api_version=api_version_reasoning,
    azure_endpoint=endpoint,
    api_key=azure_key,
)

In [3]:
reader = PdfReader("me/Amazon, 2025 February 9.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

ARTEM  MYKYTYSHYN  Toronto,  ON  |  +1  (437)  239-6115  |  artem.mykytyshyn@gmail.com |  linkedin.com/in/artem-mykytyshyn Authorized  to  work  in  Canada;  no  visa  sponsorship  required   
WORK  EXPERIENCE 
 
 
Workspace  Safety  and  Insurance  Board  (WSIB) Sep  2024  -  Present  Software  Developer  Toronto,  ON,  Canada ●  Building  an  AI-powered  document  simplification  tool  using  FastAPI/Python  and  React/JavaScript,  transforming  a  PoC  into  
a
 
secure,
 
production-ready
 
product
 
based
 
on
 
business
 
needs.
 ●  Developed  an  enterprise  request  intake  tool  from  the  ground  up  using  Spring  Boot/Java  (back  end)  and  React/JavaScript  
(front
 
end),
 
with
 
Azure
 
SQL
 
Database
 
and
 
RabbitMQ,
 
serving
 
500+
 
unique
 
users
 
and
 
supporting
 
500+
 
concurrent
 
sessions.
 ●  Identified  and  fixed  an  issue  in  a  notification  microservice,  where  a  misconfigured  message  queue  caused  mass  emails  to  be  
sent
 
company-wide
 


In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Artem Mykytyshyn"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and resume which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## Resume:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [8]:
system_prompt

"You are acting as Artem Mykytyshyn. You are answering questions on Artem Mykytyshyn's website, particularly questions related to Artem Mykytyshyn's career, background, skills and experience. Your responsibility is to represent Artem Mykytyshyn for interactions on the website as faithfully as possible. You are given a summary of Artem Mykytyshyn's background and resume which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Artem Mykytyshyn. I'm a software engineer and AI specialist.\n\n## Resume:\nARTEM  MYKYTYSHYN  Toronto,  ON  |  +1  (437)  239-6115  |  artem.mykytyshyn@gmail.com |  linkedin.com/in/artem-mykytyshyn Authorized  to  work  in  Canada;  no  visa  sponsorship  required   \nWORK  EXPERIENCE \n \n \nWorkspace  Safety  and  Insurance  Board  (WSIB) Sep  2024  -  Present  Software  Developer  Toronto,  ON,  Canada 

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai_chat.chat.completions.create(model=deployment_chat, messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [10]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [11]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [12]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and resume details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## Resume:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [13]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [14]:
# import os
# gemini = OpenAI(
#     api_key=os.getenv("GOOGLE_API_KEY"), 
#     base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
# )

In [28]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai_reasoning.beta.chat.completions.parse(model=deployment_reasoning, messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [29]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai_chat.chat.completions.create(model=deployment_chat, messages=messages)
reply = response.choices[0].message.content

In [30]:
reply

'As of now, I do not hold a patent. My focus has primarily been on software engineering and AI development, where I have been involved in various projects and contributions. If you have any questions regarding my work or projects, feel free to ask!'

In [31]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response directly and accurately addresses the user’s question, stays in character, maintains a professional tone, and invites further engagement. It meets the requirements.')

In [35]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai_chat.chat.completions.create(model=deployment_chat, messages=messages)
    return response.choices[0].message.content

In [40]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin."
        print("patent encountered")
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai_chat.chat.completions.create(model=deployment_chat, messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [41]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


patent encountered
Failed evaluation - retrying
The response is unprofessional and uses pig Latin, which is not appropriate for a professional website or client-facing interaction. It also fails to clearly address or clarify the user’s intent regarding “patent.” A better reply would politely ask what the user means by “patent” and offer relevant expertise or resources.
patent encountered
Failed evaluation - retrying
The response is unprofessional and incomprehensible – it’s written in Pig Latin and doesn’t meaningfully address the user’s request. The agent should ask for clarification in clear, professional language and offer relevant assistance regarding patents.
Passed evaluation - returning reply
